<a href="https://colab.research.google.com/github/anisoftcorporation/GenAI/blob/main/Transformers_Flan_T5_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
     torch==1.13.1 \
     torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2\
    datasets==2.11.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.17.1 requires torch

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig


# **Load the Dialog Dataset from Huggingface**



In [3]:
huggingface_dataset_name = "knkarthick/dialogsum";
dataset = load_dataset(huggingface_dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})


In [5]:
example_indices = [40,200]
dash_line = "-".join('' for x in range(5))
print(dash_line)

for i,index in enumerate(example_indices):
  print(dash_line)
  print('Example ', i+1)
  print(dash_line)
  print('INPUT DIALOG')
  print(dataset['test'][index]['dialogue'])
  print('HUMAN SUMMARY')
  print(dataset['test'][index]['summary'])
  print(dash_line)
  print()

----
----
Example  1
----
INPUT DIALOG
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
HUMAN SUMMARY
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
----

----
Example  2
----
INPUT DIALOG
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a f

# After the data has been loaded we will now import the model. Seq2Seq type model is needed here and we willbe using flan-t5-base model by Google. And we will be loading the pre-trained version of this model


In [6]:
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

The transformer model needs the texts to be tokenized for encoder and decoder. We will download the tokenizer for flan-t5-base model using AutoTokenizer.from_pretrained()

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Lets test encoding and decoding of the tokenizer

In [8]:
sample_sentence='What is my name? and what is your name?'
encoded_sentence = tokenizer(sample_sentence,return_tensors='pt') #for pytorch type tensors
print(encoded_sentence['input_ids'][0])
decoded_back =tokenizer.decode(encoded_sentence['input_ids'][0],skip_special_tokens=True) #this takes token_ids as array and the next agument removed special characters like newline or \s etc.
print(decoded_back)

tensor([363,  19,  82, 564,  58,  11, 125,  19,  39, 564,  58,   1])
What is my name? and what is your name?


Now we will run some of the sample conersations from the Dialogue dataset and run through the base LLM model , without prompt engineering or FineTuning and will observe how is it behaving.

In [9]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']
  #we will tokenize the dialogue , that will be input to the transformer's encoder
  inputs = tokenizer(dialogue,return_tensors='pt')
  generated_tokens = model.generate(inputs["input_ids"],max_new_tokens=50)
  #print(generated_tokens)
  decoded_output = tokenizer.decode(generated_tokens[0],skip_special_tokens=True)
  print(dialogue)
  print(dash_line)
  print("Human_Summary")
  print(summary)
  print("Model_generated_summary")
  print(decoded_output)

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
----
Human_Summary
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
Model_generated_summary
Person1: It's ten to nine.
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to beg

# This is not good and not even close to actual Summary. We will do prompt enginering here. we will first try Zero Shot Inference. We will convert the input as an Instruction prompt.



In [10]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']
  prompt = f"""
  Summarize the following dialogue.
  {dialogue}

  Summary:


  """

  #we will tokenize the prompt here instead of the dialogue , that will be input to the transformer's encoder
  inputs = tokenizer(prompt,return_tensors='pt')
  generated_tokens = model.generate(inputs["input_ids"],max_new_tokens=50)
  #print(generated_tokens)
  decoded_output = tokenizer.decode(generated_tokens[0],skip_special_tokens=True)
  print(f'Input Prompt:\n {prompt}')
  print(dash_line)
  print(f'Human_Summary:\n{summary}')
  print(dash_line)
  print(f'Model_generated Zero shot summary:\n{decoded_output}')


Input Prompt:
 
  Summarize the following dialogue.
  #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

  Summary:


  
----
Human_Summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
----
Model_generated Zero shot summary:
Tom is late for the train.
Input Prompt:
 
  Summarize the following dialogue.
  #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upg

Zero Shot inference is little better but not great. We will now try One shot inference. Here in Context , we will provide one example of what we expect and then we will pass one Dialogue and see how is it getting Summarized.

We will create a function that will create a prompt with one given summary and one without.

In [15]:
def make_prompt(example_indices_full, example_index_to_summarize):
  prompt = ''
  for index in example_indices_full:
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    #Now we are creating the prompt, the first part will have Dialogue and Summary, and then we will append the new Dialogue and let Model to fill in
    prompt +=f"""
    Dialogue:

    {dialogue}

    What is going on?:

    {summary}


    """
  dialogue = dataset['test'][example_index_to_summarize]['dialogue']
  prompt+=f"""
  Dialogue:

  {dialogue}

  What is going on?:


  """
  return prompt

Now use that above function to Construct One Shot prompt


In [12]:
example_indices_full = [40]
example_index_to_summarize = 200
one_shot_prompt =make_prompt(example_indices_full,example_index_to_summarize)
print(one_shot_prompt)


    Dialogue:

    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

    What is going on?:

    #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


    
    Dialogue:

    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, t

Now pass this one shot prompt to the model

In [13]:
summary = dataset['test'][example_index_to_summarize]['summary']
inputs = tokenizer(one_shot_prompt,return_tensors='pt')
generated_tokens = model.generate(inputs["input_ids"],max_new_tokens=50)
  #print(generated_tokens)
decoded_output = tokenizer.decode(generated_tokens[0],skip_special_tokens=True)

print(dash_line)
print(f'Human_Summary:\n{summary}')
print(dash_line)
print(f'Model_generated One shot summary:\n{decoded_output}')



----
Human_Summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
----
Model_generated One shot summary:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


Try few shots

In [21]:
example_indices_full = [40,80,120]
example_index_to_summarize = 20
few_shots_prompt =make_prompt(example_indices_full,example_index_to_summarize)
print(few_shots_prompt)
summary = dataset['test'][example_index_to_summarize]['summary']
inputs = tokenizer(few_shots_prompt,return_tensors='pt')
#generation_config= GenerationConfig(max_new_tokens=50)
generation_config= GenerationConfig(max_new_tokens=50,do_sample=True,temperature=1.0)
generated_tokens = model.generate(inputs["input_ids"],generation_config=generation_config)
  #print(generated_tokens)
decoded_output = tokenizer.decode(generated_tokens[0],skip_special_tokens=True)

print(dash_line)
print(f'Human_Summary:\n{summary}')
print(dash_line)
print(f'Model_generated One shot summary:\n{decoded_output}')


    Dialogue:

    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

    What is going on?:

    #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


    
    Dialogue:

    #Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic 